In [ ]:
%pip install langchain openai chromadb pandoc unstructured ebooklib sentence-transformers tiktoken ipywidgets

### 读取《神雕侠侣》

In [1]:
from langchain.document_loaders import UnstructuredEPubLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import time

timer = time.time()
loader = UnstructuredEPubLoader('../resources/shendiao.epub')
docs = loader.load()
print("load",time.time()-timer)
timer = time.time()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(docs)
print("split",time.time()-timer,len(split_docs))
timer = time.time()

embeddings=HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese')
persist_directory="../resources/chroma_db"
vectorstore = Chroma("shendiao",embeddings,persist_directory)
for i in range(len(split_docs)):
    if i%10==0:
        print("[{}] {} / {}".format(time.ctime(),i,len(split_docs)))
    vectorstore.add_documents([split_docs[i]])
print("embedding",time.time()-timer)

load 25.60806393623352
split 0.02520155906677246 1207
[Mon Oct  9 08:56:03 2023] 0 / 1207
[Mon Oct  9 08:56:06 2023] 10 / 1207
[Mon Oct  9 08:56:09 2023] 20 / 1207
[Mon Oct  9 08:56:12 2023] 30 / 1207
[Mon Oct  9 08:56:14 2023] 40 / 1207
[Mon Oct  9 08:56:17 2023] 50 / 1207
[Mon Oct  9 08:56:20 2023] 60 / 1207
[Mon Oct  9 08:56:23 2023] 70 / 1207
[Mon Oct  9 08:56:26 2023] 80 / 1207
[Mon Oct  9 08:56:29 2023] 90 / 1207
[Mon Oct  9 08:56:32 2023] 100 / 1207
[Mon Oct  9 08:56:35 2023] 110 / 1207
[Mon Oct  9 08:56:37 2023] 120 / 1207
[Mon Oct  9 08:56:40 2023] 130 / 1207
[Mon Oct  9 08:56:43 2023] 140 / 1207
[Mon Oct  9 08:56:46 2023] 150 / 1207
[Mon Oct  9 08:56:49 2023] 160 / 1207
[Mon Oct  9 08:56:52 2023] 170 / 1207
[Mon Oct  9 08:56:55 2023] 180 / 1207
[Mon Oct  9 08:56:58 2023] 190 / 1207
[Mon Oct  9 08:57:01 2023] 200 / 1207
[Mon Oct  9 08:57:03 2023] 210 / 1207
[Mon Oct  9 08:57:06 2023] 220 / 1207
[Mon Oct  9 08:57:09 2023] 230 / 1207
[Mon Oct  9 08:57:12 2023] 240 / 1207
[Mon Oc

### 《神雕侠侣》的提问

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

embeddings=HuggingFaceEmbeddings(model_name='shibing624/text2vec-base-chinese')
vectorstore = Chroma("shendiao",persist_directory="../resources/chroma_db", embedding_function=embeddings)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_base="http://3.101.17.202/v1")
chain = load_qa_chain(llm, chain_type="stuff")

query = "杨过手怎么断的？"
similar_docs = vectorstore.similarity_search(query, 3)

for d in similar_docs:
    print("======\n{}".format(d.page_content))
chain.run(input_documents=similar_docs, question=query)

C:\Users\mayan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain\llms\openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
C:\Users\mayan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\langchain\llms\openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


陆无双又问：“杨大哥。你手臂到底是怎生断的？伤势可全愈了么？”杨过道：“早就好了。是给人斩断的。”陆无双怒道：“是哪个该死的恶贼？他定然使了卑鄙的奸计，是不是？是那万恶的女魔头么？”

忽然背后一个女子声音冷笑道：“你这般背后骂人，难道便不卑鄙么？”陆无双等吃了一惊，回过头来，只见说话的是个美貌少女，正是郭芙。她手持剑柄，怒容满面，身旁男男女女站着好几个人。

陆无双奇道：“我又没骂你，我是骂那斩断杨大哥手臂的恶贼。”

刷的一响，郭芙长剑从鞘中抽出了一半，说道：“他的手臂便是我斩断的。我赔不是也赔过了，给爹爹妈妈也责罚过了，你们还在背后这般恶毒的骂我……”说到这里，眼眶一红，心中委屈无限。

※※※

原来武三通、郭芙、耶律齐、武氏兄弟等在小溪中避火，待火势弱了，才缘溪水而下，和黄蓉及完颜萍、耶律燕相遇，便到绝情谷来。一行人比一灯、杨过等早到了半日，只是在谷前谷后遍寻天竺僧和朱子柳被困之处不获，耽搁了不少时光。至于李莫愁师徒和程英姊姊进入绝情谷，却均是被周伯通童心大发而分别引来。

当下黄蓉、武三通等向一灯行礼，各人互相引见。程英从未见过黄蓉，但久闻这位师姊的大名，一直十分钦仰，当下恭恭敬敬的上前磕头，叫了声：“师姊！”黄蓉从杨过口中早知父亲暮年又收了个女徒，这时见她丰神秀美，问起父亲，得知身体安健，更是欢喜。

守在林旁的绿衣弟子见入谷外敌会合，声势甚盛，不敢出手拦阻，飞报裘千尺去了。

郭芙和陆无双怒目对视，心中互相憎恨。郭芙听母亲吩咐，竟要对程英长辈称呼，更是不喜，那一声“师叔”叫得异常勉强。

杨过和小龙女携手远远的站着。杨过向小龙女臂弯中抱着的郭襄瞧了一眼，说道：“龙儿，把这女孩儿还给她母亲罢。”小龙女举起郭襄，在她颊上亲了亲，走过去递给黄蓉，说道：“郭夫人，你的孩儿。”黄蓉称谢接过，这女孩儿自出娘胎后，直到此刻，她方始安安稳稳的抱在怀里，这份喜悦之情自是不可言喻。

杨过对郭芙朗声说道：“郭姑娘，你妹子安好无恚，我可没拿她去换救命解药。”郭芙怒道：“我妈妈来了，你自然不敢。你若无此心，抱我妹妹到此来干么？”按照杨过往日的脾性，立时便要反唇相稽，但他近月来迭遭生死大变，于这些口舌之争已不放在心上，只淡淡一笑，便和小龙女携手走开。
这时杨过头脑中也是大感晕眩，已知李莫愁运使五毒神掌时剧毒逼入掌心，适才与她手掌相交，不但剧毒传入自己体内，更传到了小龙女身上。

'杨过的手是被郭芙斩断的。'